In [11]:
import pandas as pd
import numpy as np
import warnings
import re
import math
warnings.filterwarnings("ignore")

In [22]:
chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

In [24]:
def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))


In [12]:
# подгрузка всех файлов испытаний
import os
dr = 'exp'
file_names = os.listdir(dr)

In [13]:
del file_names[-3]

In [14]:
# файлы в словарь доступ по имени файла ({имя файла: словарь датафреймов файла} )
exp_files = {}
for name in file_names:
    exp_files[name] = pd.read_excel(dr+'\\'+name, None)
#     print(name)

In [15]:
# номера плавок приводим к единому регистру (нижнему)
tmp = {}
for name in exp_files:
    for l in exp_files[name]:
        if '№ плавки' in exp_files[name][l]:
            exp_files[name][l]['№ плавки'] = exp_files[name][l]['№ плавки'].astype(str).str.lower()
        if '№ партии' in exp_files[name][l]:
            exp_files[name][l]['№ партии'] = exp_files[name][l]['№ партии'].astype(str).str.lower()
    tmp[name] = exp_files[name]
exp_files = tmp

In [16]:
# создаем единую таблицу с химией
chemistry = []
chemistry = exp_files['1квартал2017.xls'][u'Хим.анализ плавок (заготовка)']

chemistry_list_names = [
    'Хим.анализ плавок (заготовка)',
    'Хим.анализ плавок(заготовка)',
    'Хим.анализ плавок (Заготовка)',
    'Хим.анализ плавок (заготовки)' 
]

s = 0
for name, exp_file in zip(exp_files, exp_files.values()):
    for list_chem in chemistry_list_names:
        if list_chem in exp_file:
            chemistry = pd.merge(chemistry, exp_file[list_chem], how = 'outer')
            s += exp_file[list_chem].shape[0]
            break

chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)
chemistry.to_excel('chemistry.xlsx')

print(s)
print(chemistry.shape)

16057
(12306, 17)


In [17]:
chemistry = chemistry[chemistry['Поставщик']=='ЭСПК']
print(chemistry.shape)

(8091, 17)


In [25]:
for el in chem_list:
    chemistry[el]=chemistry[el].apply(ch2f)

In [53]:
def get_more_chem(x):
    for el in x:
        for i in x[el]:
            if i!=None:
                x[el] = i
    return x
                
chemistry = chemistry.groupby('№ плавки').apply(get_more_chem)

In [55]:
chemistry.drop_duplicates(inplace=True)
chemistry.shape

(7608, 17)

In [58]:
# chemistry[chemistry[['№ плавки']].duplicated(keep = False)].sort_values(['№ плавки'])

In [59]:
# подгружаем режимы
mode = pd.read_excel('all_modes_clean_2.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

(29255, 31)

In [60]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)

(29255, 47)


In [61]:
mean_chem_steel = pd.read_excel('mean_chem_steel.xlsx')

In [62]:
merge_table.shape, mean_chem_steel.shape

((29255, 47), (64, 16))

In [63]:
ls = merge_table.columns[:-15]
merge_without_chem = merge_table[merge_table['C'].isnull()]
print(merge_without_chem.shape)
merge_without_chem = merge_without_chem[ls]
merge_without_chem.shape

(7660, 47)


(7660, 32)

In [64]:
merge_table = merge_table[merge_table['C'].notnull()]
merge_table.shape

(21595, 47)

In [65]:
merge_mean_chem = pd.merge(merge_without_chem, mean_chem_steel, how='left', on='марка стали')

In [66]:
merge_mean_chem.shape

(7660, 47)

In [67]:
merge_table = pd.merge(merge_table, merge_mean_chem, how ='outer')
merge_table.shape

(29255, 47)

In [70]:
merge_table.to_excel('merge_table_mean_chem.xlsx')